In [9]:
from src.load_mta_dataset import DATASETS, download_mta_data, load_mta_data

    # Download and load MTA data
download_mta_data()
load_mta_data()

    # Print dataset information
for dataset in DATASETS:
    print(f"Dataset ID: {dataset.id}")
    print(f"Dataset Path: {dataset.path}")
    print(f"GTFS Feed: \n{dataset.gk_feed}")
    print("-" * 40)

File data/gtfs_subway.zip already exists. Skipping download.
File data/gtfsmnr.zip already exists. Skipping download.
File data/gtfs_m.zip already exists. Skipping download.


/Users/Deena/Desktop/DS 501/ds-501-cs4/.venv/lib/python3.13/site-packages/gtfs_kit/feed.py:394: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, dtype=cs.DTYPE, encoding="utf-8-sig", **csv_options)


Dataset ID: subway
Dataset Path: data/gtfs_subway.zip
GTFS Feed: 
* agency --------------------
	  agency_id                agency_name           agency_url   agency_timezone  \
0  MTA NYCT  MTA New York City Transit  http://www.mta.info  America/New_York   

  agency_lang  agency_phone  
0          en  718-330-1234  
* attributions --------------------
	None
* calendar --------------------
	  service_id  monday  tuesday  wednesday  thursday  friday  saturday  sunday  \
0     Sunday       0        0          0         0       0         0       1   
1   Saturday       0        0          0         0       0         1       0   
2    Weekday       1        1          1         1       1         0       0   

  start_date  end_date  
0   20250323  20250518  
1   20250323  20250518  
2   20250323  20250518  
* calendar_dates --------------------
	None
* fare_attributes --------------------
	None
* fare_rules --------------------
	None
* feed_info --------------------
	None
* frequencies --

In [10]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

from src.load_mta_dataset import DATASETS, download_mta_data, load_mta_data
from src.load_demographic_data import load_demographic_data, DEMOGRAPHIC_DATASETS
from src.load_census_data import load_census_data, get_area_by_fips_code

In [11]:
print(DATASETS)

[GTFSDataset(id='subway', url='https://rrgtfsfeeds.s3.amazonaws.com/gtfs_subway.zip', path=PosixPath('data/gtfs_subway.zip'), gk_feed=<gtfs_kit.feed.Feed object at 0x16016d6a0>), GTFSDataset(id='metro_north_railroad', url='https://rrgtfsfeeds.s3.amazonaws.com/gtfsmnr.zip', path=PosixPath('data/gtfsmnr.zip'), gk_feed=<gtfs_kit.feed.Feed object at 0x16016c160>), GTFSDataset(id='manhattan_bus', url='https://rrgtfsfeeds.s3.amazonaws.com/gtfs_m.zip', path=PosixPath('data/gtfs_m.zip'), gk_feed=<gtfs_kit.feed.Feed object at 0x15fc01e50>)]


In [12]:
subway_data = DATASETS[0]
subway_data.gk_feed.describe()

,indicator,value
0,agencies,[MTA New York City Transit]
1,timezone,America/New_York
2,start_date,20250323
3,end_date,20250518
4,num_routes,30
5,num_trips,20298
6,num_stops,1497
7,num_shapes,311
8,sample_date,20250327
9,num_routes_active_on_sample_date,28


In [6]:
stops_df = subway_data.gk_feed.stops
stops_gdf = gpd.GeoDataFrame(
    stops_df,
    geometry=gpd.points_from_xy(stops_df["stop_lon"], stops_df["stop_lat"]),
    crs="EPSG:4326"
)

In [13]:
from pathlib import Path
COMMUNITY_DISTRICTS_GEOJSON = Path("demographic_data/community_districts.geojson")
community_districts = gpd.read_file(COMMUNITY_DISTRICTS_GEOJSON)
community_districts = community_districts.to_crs("EPSG:4326") 

In [19]:
from src.load_demographic_data import load_demographic_data, DEMOGRAPHIC_DATASETS
load_demographic_data()
total_population_data = DEMOGRAPHIC_DATASETS[1]

Loaded Median Incomes from demographic_data/median_incomes.csv
Loaded Total Population from demographic_data/total_population.csv
Loaded Total Population by Race Ethnicity from demographic_data/total_population_by_race_ethnicity.csv
Loaded Total Population by Age Group from demographic_data/total_population_by_age_group.csv


In [20]:
# Get the most recent timeframe for the median income dataset
most_recent_timeframe = total_population_data.dataframe["TimeFrame"].max()
print(f"Most recent timeframe: {most_recent_timeframe}")

# Make sure that there is a row for the most recent timeframe for all locations
recent_timeframe_rows = total_population_data.dataframe[
    total_population_data.dataframe["TimeFrame"] == most_recent_timeframe
]
assert total_population_data.dataframe["Location"].nunique() == recent_timeframe_rows["Location"].nunique(), \
    "Not all locations have a row for the most recent timeframe in the median income dataset."

filtered_total_population = recent_timeframe_rows

# Check the first few rows of the filtered total population dataset
print(filtered_total_population.head())

Most recent timeframe: 2022
                       Location  TimeFrame DataFormat      Data Fips
2890       Battery Park/Tribeca       2022     Number   68418.0  101
2891          Greenwich Village       2022     Number   80684.0  102
2892            Lower East Side       2022     Number  141787.0  103
2893            Chelsea/Clinton       2022     Number  112582.0  104
2894  Midtown Business District       2022     Number   52570.0  105


In [27]:
filtered_total_population = filtered_total_population.set_index("Location")

In [28]:
import pandas as pd
filtered_total_population["Data"] = pd.to_numeric(filtered_total_population["Data"], errors="coerce")

In [29]:
from src.load_census_data import load_census_data, get_area_by_fips_code, GEOJSON_DATASET

# Load census data
load_census_data()

fips = filtered_total_population["Fips"]
areas = get_area_by_fips_code(fips)
print(areas)

# Convert the Fips column to integers
filtered_total_population["Fips"] = filtered_total_population["Fips"].astype(int)
print(filtered_total_population["Fips"].dtype)
print(areas["Fips"].dtype)
# For filtered total population, join with the areas on Fips
filtered_total_population = filtered_total_population.join(areas.set_index("Fips"), on="Fips")


# plt.figure(figsize=(8,6))
# sns.regplot(data=df, x='Pop_Density', y='Transit_Stops', ci=None)
# plt.title(f"X vs Population Density")
# plt.xlabel("X")
# plt.ylabel("Population Density")
# plt.tight_layout()
# plt.show()

# # Add trendline
# z_filtered = np.polyfit(
#     x,
#     y,
#     1,
# )
# p_filtered = np.poly1d(z_filtered)
# plt.plot(
#     x,
#     p_filtered(x),
#     color="red",
#     label="Trendline"
# )
# plt.legend()

# # Display the correlation coefficient
# correlation_coefficient_filtered = x.corr(y)
# plt.text(
#     0.05,
#     0.95,
#     f"Correlation Coefficient: {correlation_coefficient_filtered:.2f}",
#     transform=plt.gca().transAxes,
#     fontsize=12,
#     verticalalignment="top",
# )

# plt.show()

    Fips  Area (km^2)
0    101     7.224043
1    102     6.119568
2    103     7.579787
3    104     7.758009
4    105     6.365355
5    106     6.956614
6    107     8.792266
7    108     9.449743
8    109     6.897338
9    110     6.183471
10   111    10.704405
11   112    12.629904
12   201    12.354868
13   202     9.922601
14   203     7.464240
15   204     9.358614
16   205     6.429054
17   206     7.111113
18   207     9.491898
19   208    13.452351
20   209    18.185304
21   210    27.978013
22   211    16.425201
23   212    24.340185
24   301    21.229758
25   302    12.234741
26   303    12.598806
27   304     9.129683
28   305    26.641602
29   306    13.954589
30   307    16.987039
31   308     7.560612
32   309     7.098861
33   310    17.561566
34   311    16.559264
35   312    16.254589
36   313    14.635916
37   314    12.730436
38   315    20.184914
39   316     8.192347
40   317    15.089275
41   318    40.063597
42   401    25.163524
43   402    21.192367
44   403  

/Users/Deena/Desktop/DS 501/ds-501-cs4/.venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


ValueError: columns overlap but no suffix specified: Index(['Area (km^2)'], dtype='object')